


# **SKELETON BASED ACTION RECOGNITION**

**Problem Statement**

To Develop and Evaluate a machine learning model for a 3D skeleton-based action recognition system using UTD MHAD dataset.



**Feature Extraction**

To Calculate the Euclidean distance between Shoulder center joint to remaining all joints.

In [8]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.io import loadmat
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from pycaret.classification import *
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [6]:
pip install pycaret

     |████████████████████████████████| 266 kB 28.2 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 80 kB 8.4 MB/s 
     |████████████████████████████████| 261 kB 54.9 MB/s 
     |████████████████████████████████| 167 kB 49.5 MB/s 
     |████████████████████████████████| 1.3 MB 55.7 MB/s 
     |████████████████████████████████| 14.6 MB 123 kB/s 
     |████████████████████████████████| 111 kB 72.8 MB/s 
     |████████████████████████████████| 1.7 MB 47.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0 MB 37.7 MB/s 
     |████████████████████████████████| 271 kB 74.5 MB/s 
     |████████████████████████████████| 6.8 MB 60.0 MB/s 
     |████████████████████████████████| 636 kB 66.1 MB/s 
     |████████████████████████████████| 675 kB 73.1 MB/s 
     |█████████████████

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
train = []
test = []
dataDir = "/content/gdrive/MyDrive/Skeleton/"
for file in os.listdir(dataDir):
    if file.endswith(".mat"):
        if "s2" in file or "s4" in file or "s6" in file or "s8" in file:
            train.append([loadmat(dataDir+file)["d_skel"],file.split("_")[0]])
        else:
            test.append([loadmat(dataDir+file)["d_skel"],file.split("_")[0]])

KeyboardInterrupt: ignored

In [ ]:
def trim(x,minvalue):
    r = (x.shape[2]/(minvalue) )
    array = []
    t=0+r
    i=int(t)
    # print(x.shape[2])
    while i+1 < x.shape[2]:
        array.append( x[:,:,i] + ( (x[:,:,i] - x[:,:,i+1])*(t-i) )  )
        t+=r
        i=int(t)-1
    return np.array(array)[:40].transpose(1,2,0)

In [ ]:
print(test[-1][0].shape)
print(trim(test[-1][0],41).shape)

(20, 3, 67)
(20, 3, 40)


In [ ]:
d = lambda x1,y1,z1,x2,y2,z2 : ((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)**0.5
def dist_func(skl):
    dvals = d(skl[:,0,:],skl[:,1,:],skl[:,2,:],skl[1,0,:],skl[1,1,:],skl[1,2,:])
    dvals = (dvals[:]-dvals.mean())/dvals.std()
    return dvals

In [ ]:
x = train[0][0][:,0,0]
y = train[0][0][:,1,0]
z = train[0][0][:,2,0]

d(x,y,z,x[1],y[1],z[1])

In [ ]:
np.array([dist_func(train[0][0])[i][0] for i in range(20)])

In [ ]:
def return_distance_and_action(mats):
    x = []
    y = []
    for skl in mats:
        x.append(dist_func(trim(skl[0],41)).flatten())
        y.append(skl[1])
    return np.array(x),np.array(y)

In [ ]:
X_train,y_train = return_distance_and_action(train)
X_test,y_test = return_distance_and_action(test)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("Accuracy: ",(accuracy_score(y_test, y_pred)))

sns.heatmap(confusion_matrix(y_test,y_pred))

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))

sns.heatmap(confusion_matrix(y_test,y_pred))

In [ ]:
mnb = MultinomialNB()
ms = MinMaxScaler()
mnb.fit(ms.fit_transform(X_train), y_train)
y_pred = mnb.predict(ms.fit_transform(X_test))
print("Accuracy: ",accuracy_score(y_test,y_pred))

sns.heatmap(confusion_matrix(y_test,y_pred))

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))

sns.heatmap(confusion_matrix(y_test,y_pred))

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))

sns.heatmap(confusion_matrix(y_test,y_pred))

In [ ]:
etc = ExtraTreesClassifier()
etc.fit(X_train, y_train)
y_pred = etc.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))

sns.heatmap(confusion_matrix(y_test,y_pred))

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))

sns.heatmap(confusion_matrix(y_test,y_pred))

In [ ]:
lightgbm = LGBMClassifier()
lightgbm.fit(X_train, y_train)
y_pred = lightgbm.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))

sns.heatmap(confusion_matrix(y_test,y_pred))